# Prompt, Mask, Train, Save, Pipeline-Predict

#### prompt template: 2

#### joint tasks: component classification, linkage classification

# Notebook:

##### Imports
##### replace 'MajorClaim' by 'Stance'
##### make train and test datasets
##### train BERT Mask on the train dataset
##### save the model
##### create a pipeline using the saved model
##### run predictions on it
##### evaluate with the test set

### Imports

In [1]:
!pip install transformers
!pip install ipywidgets
!pip install IProgress
!pip install pandas
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import transformers
import torch

In [3]:
from transformers import BertTokenizer, BertForMaskedLM

import numpy as np

import datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import load_metric

In [4]:
import pandas as pd

### tokenizer

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

### make the train, test dataframe

In [6]:
df = pd.read_pickle("/notebooks/Prompting/notebooks/mask_joint_tasks/pe_dataset_w_prompts_1_2_3_joint_corrected_pickle")

In [7]:
# rename MajorClaim as stance

In [8]:
df

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_2,prompted_representation_3,prompted_representation_2_correct,prompted_representation_2_joint
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
1,essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,Topic: Should students be taught to compete or...,1.00,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADV, ADJ, NOUN, ADP,...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5968,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...
5969,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VER

In [9]:
# no need for renaming here cause it's alrady done

In [10]:
# rename NotLinked to separate

In [11]:
def notlinked_separate(x):
    
    prompted_rep = x.prompted_representation_2_joint
    
    last_token = prompted_rep.split()[-1]
    
    if last_token == 'NotLinked.':
    
        new_rep = prompted_rep.replace(last_token, "Separate.")
        
    else:
        
        new_rep = prompted_rep
    
    return new_rep
    
    

In [12]:
df['corrected_representation_2'] = df.apply(lambda x: notlinked_separate(x), axis=1)

In [13]:
# now do a sanity check to see if it's correct

In [14]:
last_tokens_list_link = []

In [15]:
def get_last_token_link(x):
    
    prompted_rep = x.corrected_representation_2
    
    last_token = prompted_rep.split()[-1]
    
    last_tokens_list_link.append(last_token)

In [16]:
df.apply(lambda x: get_last_token_link(x), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
5968    None
5969    None
5970    None
5971    None
5972    None
Length: 5973, dtype: object

In [17]:
last_tokens_list_link

['Linked.',
 'Linked.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Linked.',
 'Linked.',
 'Separate.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Separate.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Linked.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Linked.',
 'Separate.',
 'Linked.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Linked.',
 'Linked.',
 'Linked.',
 'Linked.',
 'Linked.',
 'Separate.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Linked.',
 'Separate.',
 'Linked.',
 'Linked.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Separate.',
 'Separate.',


In [18]:
set(last_tokens_list_link)

{'Linked.', 'Separate.'}

In [19]:
####### YAAAAAAAAASSSSSSSSSS KINGGGGGGGGGGGGGGGG # correct.

In [20]:
# now the component class

In [21]:
# df['prompted_representation_2_joint'][0].split()[-5]

In [22]:
last_tokens_list_comp = []

In [23]:
def get_last_token_comp(x):
    
    prompted_rep = x.prompted_representation_2_joint
    
    last_token = prompted_rep.split()[-5]
    
    last_tokens_list_comp.append(last_token)

In [24]:
df.apply(lambda x: get_last_token_comp(x), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
5968    None
5969    None
5970    None
5971    None
5972    None
Length: 5973, dtype: object

In [25]:
last_tokens_list_comp

['Stance.',
 'Stance.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Stance.',
 'Stance.',
 'Premise.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Stance.',
 'Stance.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Stance.',
 'Stance.',
 'Claim.',
 'Claim.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Stance.',
 'Stance.',
 'Claim.',
 'Claim.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Claim.',
 'Stance.',
 'Stance.',
 'Claim.',
 'Claim.',
 'Claim.',
 'Claim.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Premise.',
 'Stance.',
 'Claim.',
 'Premise.',
 'Premise.',


In [26]:
set(last_tokens_list_comp)

{'Claim.', 'Premise.', 'Stance.'}

In [27]:
######### YASSSSSSSSSSS JJJJJJJJJOINTTTTTTTTTTT KINGGGGGGGGGGGGGGGG

## make train, test datasets

### train dataset

#### everything we do for the [-1] (which is link/notlinked now), we must do for the component class

In [28]:
df_train = df[df.split == 'TRAIN']

In [29]:
df_train = df_train.reset_index(drop=True)

In [30]:
df_train

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_2,prompted_representation_3,prompted_representation_2_correct,prompted_representation_2_joint,corrected_representation_2
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
1,essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,1.00,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADV, ADJ, NOUN, ADP,...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4708,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
4709,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying h

In [31]:
df_train.split.value_counts()

TRAIN    4713
Name: split, dtype: int64

In [32]:
prompted_texts_train_full = df_train['corrected_representation_2'][:].tolist()

In [33]:
len(prompted_texts_train_full)

4713

In [34]:
import random

In [35]:
prompted_texts_val = random.sample(prompted_texts_train_full, 472)

In [36]:
len(prompted_texts_val)

472

In [37]:
prompted_texts_train = [x for x in prompted_texts_train_full if x not in prompted_texts_val]

In [38]:
len(prompted_texts_train)

4237

In [39]:
len(prompted_texts_train) + len(prompted_texts_val)

4709

In [40]:
### prepare train dataset

In [41]:
inputs_train = tokenizer(prompted_texts_train, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [42]:
inputs_train

{'input_ids': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [43]:
inputs_train['input_ids'].shape

torch.Size([4237, 512])

In [44]:
# for training we will need labels

In [45]:
inputs_train['labels'] = inputs_train.input_ids.detach().clone()

In [46]:
inputs_train

{'input_ids': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
      

In [47]:
inputs_train['labels'].shape

torch.Size([4237, 512])

In [48]:
# find where the last 102 is

In [49]:
(inputs_train['input_ids'][3] == 102).nonzero(as_tuple=True)[0].item()

194

In [50]:
inputs_train['input_ids'][3][236]

tensor(0)

In [51]:
tokenizer.decode(18458)

'p r e m i s e'

In [52]:
inputs_train['input_ids'][0][143]

tensor(1012)

In [53]:
tokenizer.decode(1012)

'.'

In [54]:
tokenizer.decode(5799)

'l i n k e d'

In [55]:
tokenizer.decode(4366)

'c l a i m'

In [56]:
tokenizer.decode(18458)

'p r e m i s e'

In [57]:
# claim = 4366, premise = 18458

In [58]:
(inputs_train['input_ids'][1] == 102).nonzero(as_tuple=True)[0].item()

169

In [59]:
inputs_train['input_ids'][1][149]

tensor(1010)

In [60]:
# check if idx[sep - 2] is linked/separate class for all

In [61]:
sep_indices_list = []

for i in range(len(prompted_texts_train)):
    sep_idx = (inputs_train['input_ids'][i] == 102).nonzero(as_tuple=True)[0].item()
    sep_indices_list.append(sep_idx)

In [62]:
sep_indices_list

[144,
 169,
 168,
 194,
 243,
 173,
 124,
 153,
 311,
 137,
 163,
 177,
 166,
 211,
 162,
 172,
 147,
 172,
 142,
 152,
 199,
 165,
 158,
 185,
 166,
 128,
 163,
 202,
 154,
 142,
 231,
 164,
 151,
 148,
 154,
 244,
 230,
 176,
 159,
 142,
 147,
 180,
 150,
 155,
 167,
 159,
 221,
 193,
 171,
 193,
 166,
 193,
 188,
 150,
 127,
 144,
 133,
 115,
 134,
 156,
 178,
 132,
 154,
 123,
 147,
 121,
 151,
 179,
 125,
 149,
 163,
 151,
 200,
 135,
 175,
 140,
 156,
 138,
 149,
 128,
 136,
 139,
 180,
 186,
 140,
 147,
 122,
 184,
 139,
 153,
 183,
 153,
 141,
 140,
 125,
 138,
 132,
 119,
 153,
 254,
 238,
 125,
 133,
 150,
 168,
 167,
 193,
 173,
 142,
 139,
 142,
 125,
 130,
 165,
 124,
 118,
 209,
 156,
 177,
 153,
 155,
 147,
 151,
 173,
 118,
 176,
 146,
 162,
 145,
 150,
 172,
 128,
 172,
 139,
 142,
 178,
 111,
 127,
 121,
 128,
 170,
 135,
 188,
 169,
 174,
 173,
 219,
 160,
 127,
 168,
 168,
 200,
 159,
 168,
 149,
 151,
 197,
 150,
 161,
 212,
 142,
 153,
 232,
 147,
 170,
 141,
 123

In [63]:
len(sep_indices_list)

4237

In [64]:
link_tokens_indices_list = [x - 2 for x in sep_indices_list]

In [65]:
link_tokens_indices_list

[142,
 167,
 166,
 192,
 241,
 171,
 122,
 151,
 309,
 135,
 161,
 175,
 164,
 209,
 160,
 170,
 145,
 170,
 140,
 150,
 197,
 163,
 156,
 183,
 164,
 126,
 161,
 200,
 152,
 140,
 229,
 162,
 149,
 146,
 152,
 242,
 228,
 174,
 157,
 140,
 145,
 178,
 148,
 153,
 165,
 157,
 219,
 191,
 169,
 191,
 164,
 191,
 186,
 148,
 125,
 142,
 131,
 113,
 132,
 154,
 176,
 130,
 152,
 121,
 145,
 119,
 149,
 177,
 123,
 147,
 161,
 149,
 198,
 133,
 173,
 138,
 154,
 136,
 147,
 126,
 134,
 137,
 178,
 184,
 138,
 145,
 120,
 182,
 137,
 151,
 181,
 151,
 139,
 138,
 123,
 136,
 130,
 117,
 151,
 252,
 236,
 123,
 131,
 148,
 166,
 165,
 191,
 171,
 140,
 137,
 140,
 123,
 128,
 163,
 122,
 116,
 207,
 154,
 175,
 151,
 153,
 145,
 149,
 171,
 116,
 174,
 144,
 160,
 143,
 148,
 170,
 126,
 170,
 137,
 140,
 176,
 109,
 125,
 119,
 126,
 168,
 133,
 186,
 167,
 172,
 171,
 217,
 158,
 125,
 166,
 166,
 198,
 157,
 166,
 147,
 149,
 195,
 148,
 159,
 210,
 140,
 151,
 230,
 145,
 168,
 139,
 121

In [66]:
list_index_minus_2 = []

for idx, val in enumerate(link_tokens_indices_list):
    at_idx_minus_2 = inputs_train['input_ids'][idx][val].item()
    list_index_minus_2.append(at_idx_minus_2)

In [67]:
set(list_index_minus_2)

{3584, 5799}

In [68]:
tokenizer.decode(3584)

's e p a r a t e'

In [69]:
tokenizer.decode(5799)

'l i n k e d'

In [70]:
# confirm that all at [len-7] indices are either stance, claim or premise

In [71]:
class_tokens_indices_list = [x - 7 for x in sep_indices_list]

In [72]:
class_tokens_indices_list

[137,
 162,
 161,
 187,
 236,
 166,
 117,
 146,
 304,
 130,
 156,
 170,
 159,
 204,
 155,
 165,
 140,
 165,
 135,
 145,
 192,
 158,
 151,
 178,
 159,
 121,
 156,
 195,
 147,
 135,
 224,
 157,
 144,
 141,
 147,
 237,
 223,
 169,
 152,
 135,
 140,
 173,
 143,
 148,
 160,
 152,
 214,
 186,
 164,
 186,
 159,
 186,
 181,
 143,
 120,
 137,
 126,
 108,
 127,
 149,
 171,
 125,
 147,
 116,
 140,
 114,
 144,
 172,
 118,
 142,
 156,
 144,
 193,
 128,
 168,
 133,
 149,
 131,
 142,
 121,
 129,
 132,
 173,
 179,
 133,
 140,
 115,
 177,
 132,
 146,
 176,
 146,
 134,
 133,
 118,
 131,
 125,
 112,
 146,
 247,
 231,
 118,
 126,
 143,
 161,
 160,
 186,
 166,
 135,
 132,
 135,
 118,
 123,
 158,
 117,
 111,
 202,
 149,
 170,
 146,
 148,
 140,
 144,
 166,
 111,
 169,
 139,
 155,
 138,
 143,
 165,
 121,
 165,
 132,
 135,
 171,
 104,
 120,
 114,
 121,
 163,
 128,
 181,
 162,
 167,
 166,
 212,
 153,
 120,
 161,
 161,
 193,
 152,
 161,
 142,
 144,
 190,
 143,
 154,
 205,
 135,
 146,
 225,
 140,
 163,
 134,
 116

In [73]:
# inputs_pe['input_ids'][0][143].item() # correct

In [74]:
# now get a list of input ids at all these indics: they should be either claim = 4366, premise = 18458

In [75]:
list_index_minus_7 = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_7 = inputs_train['input_ids'][idx][val].item()
    list_index_minus_7.append(at_idx_minus_7)

In [76]:
set(list_index_minus_7)

{4366, 11032, 18458}

In [77]:
### ok. linked index is sep-2, component index is sep-7

In [78]:
tokenizer.decode(11032)

's t a n c e'

In [79]:
# correct

In [80]:
# now make the class token indices 103 # we will need two lists now

In [81]:
for idx, val in enumerate(class_tokens_indices_list):
    inputs_train['input_ids'][idx][val] = 103

In [82]:
for idx, val in enumerate(link_tokens_indices_list):
    inputs_train['input_ids'][idx][val] = 103

In [83]:
# now check if they are 103

In [84]:
unsontroi_check_list_class = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_train['input_ids'][idx][val].item()
    unsontroi_check_list_class.append(at_idx_minus_2)

In [85]:
set(unsontroi_check_list_class)

{103}

In [86]:
unsontroi_check_list_link = []

for idx, val in enumerate(link_tokens_indices_list):
    at_idx_minus_2 = inputs_train['input_ids'][idx][val].item()
    unsontroi_check_list_link.append(at_idx_minus_2)

In [87]:
set(unsontroi_check_list_link)

{103}

In [88]:
# correct

In [89]:
class PeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [90]:
train_dataset = PeDataset(inputs_train)

In [91]:
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=)
# maybe we don't need this.

### prepare val dataset

In [92]:
inputs_eval = tokenizer(prompted_texts_val, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [93]:
inputs_eval

{'input_ids': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [94]:
inputs_eval['input_ids'].shape

torch.Size([472, 512])

In [95]:
# for training we will need labels

In [96]:
inputs_eval['labels'] = inputs_eval.input_ids.detach().clone()

In [97]:
inputs_eval

{'input_ids': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
      

In [98]:
inputs_eval['labels'].shape

torch.Size([472, 512])

In [99]:
# find where the last 102 is

In [100]:
(inputs_eval['input_ids'][0] == 102).nonzero(as_tuple=True)[0].item()

150

In [101]:
tokenizer.decode(4366)

'c l a i m'

In [102]:
tokenizer.decode(18458)

'p r e m i s e'

In [103]:
# claim = 4366, premise = 18458

In [104]:
(inputs_eval['input_ids'][1] == 102).nonzero(as_tuple=True)[0].item()

127

In [105]:
inputs_eval['input_ids'][1][130]

tensor(0)

In [106]:
# confirm that all at [len-2] indices are either claim or premise

In [107]:
sep_indices_list = []

for i in range(len(prompted_texts_val)):
    sep_idx = (inputs_eval['input_ids'][i] == 102).nonzero(as_tuple=True)[0].item()
    sep_indices_list.append(sep_idx)

In [108]:
sep_indices_list

[150,
 127,
 174,
 153,
 152,
 145,
 179,
 184,
 168,
 161,
 152,
 123,
 151,
 126,
 125,
 150,
 203,
 148,
 185,
 152,
 217,
 148,
 153,
 138,
 151,
 148,
 135,
 145,
 145,
 160,
 158,
 129,
 153,
 173,
 144,
 148,
 152,
 165,
 195,
 171,
 158,
 146,
 173,
 122,
 143,
 197,
 190,
 153,
 158,
 153,
 142,
 182,
 166,
 210,
 170,
 142,
 172,
 142,
 190,
 169,
 157,
 162,
 157,
 227,
 193,
 174,
 210,
 181,
 150,
 165,
 152,
 145,
 181,
 120,
 169,
 159,
 158,
 135,
 152,
 222,
 158,
 192,
 163,
 148,
 203,
 185,
 159,
 124,
 175,
 151,
 140,
 128,
 167,
 141,
 138,
 186,
 147,
 158,
 137,
 143,
 184,
 142,
 141,
 176,
 148,
 162,
 184,
 137,
 160,
 189,
 145,
 180,
 130,
 124,
 139,
 142,
 174,
 134,
 131,
 175,
 125,
 159,
 142,
 130,
 131,
 149,
 162,
 157,
 185,
 181,
 140,
 189,
 128,
 131,
 179,
 135,
 251,
 140,
 147,
 150,
 124,
 129,
 181,
 139,
 203,
 132,
 173,
 172,
 159,
 186,
 218,
 169,
 139,
 175,
 171,
 164,
 193,
 168,
 132,
 131,
 142,
 168,
 134,
 162,
 171,
 163,
 137

In [109]:
len(sep_indices_list)

472

In [110]:
link_tokens_indices_list = [x - 2 for x in sep_indices_list]

In [111]:
link_tokens_indices_list

[148,
 125,
 172,
 151,
 150,
 143,
 177,
 182,
 166,
 159,
 150,
 121,
 149,
 124,
 123,
 148,
 201,
 146,
 183,
 150,
 215,
 146,
 151,
 136,
 149,
 146,
 133,
 143,
 143,
 158,
 156,
 127,
 151,
 171,
 142,
 146,
 150,
 163,
 193,
 169,
 156,
 144,
 171,
 120,
 141,
 195,
 188,
 151,
 156,
 151,
 140,
 180,
 164,
 208,
 168,
 140,
 170,
 140,
 188,
 167,
 155,
 160,
 155,
 225,
 191,
 172,
 208,
 179,
 148,
 163,
 150,
 143,
 179,
 118,
 167,
 157,
 156,
 133,
 150,
 220,
 156,
 190,
 161,
 146,
 201,
 183,
 157,
 122,
 173,
 149,
 138,
 126,
 165,
 139,
 136,
 184,
 145,
 156,
 135,
 141,
 182,
 140,
 139,
 174,
 146,
 160,
 182,
 135,
 158,
 187,
 143,
 178,
 128,
 122,
 137,
 140,
 172,
 132,
 129,
 173,
 123,
 157,
 140,
 128,
 129,
 147,
 160,
 155,
 183,
 179,
 138,
 187,
 126,
 129,
 177,
 133,
 249,
 138,
 145,
 148,
 122,
 127,
 179,
 137,
 201,
 130,
 171,
 170,
 157,
 184,
 216,
 167,
 137,
 173,
 169,
 162,
 191,
 166,
 130,
 129,
 140,
 166,
 132,
 160,
 169,
 161,
 135

In [112]:
list_index_minus_2 = []

for idx, val in enumerate(link_tokens_indices_list):
    at_idx_minus_2 = inputs_eval['input_ids'][idx][val].item()
    list_index_minus_2.append(at_idx_minus_2)

In [113]:
set(list_index_minus_2)

{3584, 5799}

In [114]:
tokenizer.decode(3584)

's e p a r a t e'

In [115]:
tokenizer.decode(5799)

'l i n k e d'

In [116]:
# confirm that all at [len-7] indices are either stance, claim or premise

In [117]:
class_tokens_indices_list = [x - 7 for x in sep_indices_list]

In [118]:
class_tokens_indices_list

[143,
 120,
 167,
 146,
 145,
 138,
 172,
 177,
 161,
 154,
 145,
 116,
 144,
 119,
 118,
 143,
 196,
 141,
 178,
 145,
 210,
 141,
 146,
 131,
 144,
 141,
 128,
 138,
 138,
 153,
 151,
 122,
 146,
 166,
 137,
 141,
 145,
 158,
 188,
 164,
 151,
 139,
 166,
 115,
 136,
 190,
 183,
 146,
 151,
 146,
 135,
 175,
 159,
 203,
 163,
 135,
 165,
 135,
 183,
 162,
 150,
 155,
 150,
 220,
 186,
 167,
 203,
 174,
 143,
 158,
 145,
 138,
 174,
 113,
 162,
 152,
 151,
 128,
 145,
 215,
 151,
 185,
 156,
 141,
 196,
 178,
 152,
 117,
 168,
 144,
 133,
 121,
 160,
 134,
 131,
 179,
 140,
 151,
 130,
 136,
 177,
 135,
 134,
 169,
 141,
 155,
 177,
 130,
 153,
 182,
 138,
 173,
 123,
 117,
 132,
 135,
 167,
 127,
 124,
 168,
 118,
 152,
 135,
 123,
 124,
 142,
 155,
 150,
 178,
 174,
 133,
 182,
 121,
 124,
 172,
 128,
 244,
 133,
 140,
 143,
 117,
 122,
 174,
 132,
 196,
 125,
 166,
 165,
 152,
 179,
 211,
 162,
 132,
 168,
 164,
 157,
 186,
 161,
 125,
 124,
 135,
 161,
 127,
 155,
 164,
 156,
 130

In [119]:
# inputs_pe['input_ids'][0][143].item() # correct

In [120]:
# now get a list of input ids at all these indics: they should be either claim = 4366, premise = 18458

In [121]:
list_index_minus_7 = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_7 = inputs_eval['input_ids'][idx][val].item()
    list_index_minus_7.append(at_idx_minus_7)

In [122]:
set(list_index_minus_7)

{4366, 11032, 18458}

In [123]:
### ok. linked index is sep-2, component index is sep-7

In [124]:
tokenizer.decode(11032)

's t a n c e'

In [125]:
# correct

In [126]:
# now make the class token indices 103 # we will need two lists now

In [127]:
for idx, val in enumerate(class_tokens_indices_list):
    inputs_eval['input_ids'][idx][val] = 103

In [128]:
for idx, val in enumerate(link_tokens_indices_list):
    inputs_eval['input_ids'][idx][val] = 103

In [129]:
# now check if they are 103

In [130]:
unsontroi_check_list_class = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_eval['input_ids'][idx][val].item()
    unsontroi_check_list_class.append(at_idx_minus_2)

In [137]:
set(unsontroi_check_list_class)

{103}

In [138]:
unsontroi_check_list_link = []

for idx, val in enumerate(link_tokens_indices_list):
    at_idx_minus_2 = inputs_eval['input_ids'][idx][val].item()
    unsontroi_check_list_link.append(at_idx_minus_2)

In [139]:
set(unsontroi_check_list_link)

{103}

In [140]:
# correct

In [141]:
# class PeDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings
#     def __getitem__(self, idx):
#         return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#     def __len__(self):
#         return len(self.encodings.input_ids)

In [142]:
eval_dataset = PeDataset(inputs_eval)

### test dataset

##### do we need this? not for now

In [137]:
df_test = df[df.split == 'TEST']

In [138]:
df_test = df_test.reset_index(drop=True)

In [139]:
df_test

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_2,prompted_representation_3,prompted_representation_2_correct,prompted_representation_2_joint,corrected_representation_2
0,essay004,T1,MajorClaim 262 376,this industry has affected the cultural attrib...,MajorClaim,MajorClaim,[],,Linked,TEST,...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, DET...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
1,essay004,T2,MajorClaim 1663 1758,the tourism has created threatening pressure o...,MajorClaim,MajorClaim,[],,Linked,TEST,...,1.00,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, VER...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
2,essay004,T3,Claim 179 239,the tourism bring large profit for the destina...,Claim,Claim,[],Attack,NotLinked,TEST,...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, VERB, ADJ, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
3,essay004,T4,Claim 953 1031,international tourism can create negative impa...,Claim,Claim,[],Support,Linked,TEST,...,0.50,0,Topic: International tourism is now more commo...,"Part Of Speech tags: ADJ, NOUN, VERB, VERB, AD...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
4,essay004,T5,Claim 1578 1624,tourism has threatened the nature environments,Claim,Claim,[],Support,Linked,TEST,...,0.75,0,Topic: International tourism is now more commo...,"Part Of Speech tags: NOUN, AUX, VERB, DET, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,essay398,T12,Claim 1484 1589,universities should encourage more girls to ch...,Claim,Claim,[],Support,Linked,TEST,...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: NOUN, VERB, VERB, ADJ, NO...",Topic: We can not forcedly put the same number...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
1256,essay398,T13,Premise 1595 1648,this could avoid imbalance of gender in some s...,Premise,Premise,[],Support,NotLinked,TEST,...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: DET, VERB, VERB, NOUN, AD...",Topic: We can not forcedly put the same number...,Which of these cho

In [140]:
df_test.split.value_counts()

TEST    1260
Name: split, dtype: int64

In [141]:
prompted_texts_test = df_test['prompted_rep_3_w_modif_MC_corrected'][:].tolist()

KeyError: 'prompted_rep_3_w_modif_MC_corrected'

In [ ]:
len(prompted_texts_test)

In [ ]:
inputs_test = tokenizer(prompted_texts_test, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [ ]:
inputs_test

In [ ]:
inputs_test['input_ids'].shape

In [ ]:
# for test we do not need labels

In [ ]:
#inputs_train['labels'] = inputs_train.input_ids.detach().clone()

In [ ]:
#inputs_train

In [ ]:
#inputs_train['labels'].shape

In [ ]:
# find where the last 102 is

In [ ]:
#(inputs_train['input_ids'][0] == 102).nonzero(as_tuple=True)[0].item()

In [ ]:
#tokenizer.decode(4366)

In [ ]:
#tokenizer.decode(18458)

In [ ]:
# claim = 4366, premise = 18458

In [ ]:
#(inputs_train['input_ids'][1] == 102).nonzero(as_tuple=True)[0].item()

In [ ]:
#inputs_train['input_ids'][1][169]

In [ ]:
# confirm that all at [len-2] indices are either claim or premise

In [ ]:
sep_indices_list_test = []

for i in range(len(prompted_texts_test)):
    sep_idx = (inputs_test['input_ids'][i] == 102).nonzero(as_tuple=True)[0].item()
    sep_indices_list_test.append(sep_idx)

In [ ]:
sep_indices_list_test

In [ ]:
len(sep_indices_list_test)

In [ ]:
class_tokens_indices_test_list = [x - 2 for x in sep_indices_list_test]

In [ ]:
class_tokens_indices_test_list

In [ ]:
# inputs_pe['input_ids'][0][143].item() # correct

In [ ]:
# now get a list of input ids at all these indices: they should be either claim = 4366, premise = 18458

In [ ]:
list_index_minus_2_test = []

for idx, val in enumerate(class_tokens_indices_test_list):
    at_idx_minus_2 = inputs_test['input_ids'][idx][val].item()
    list_index_minus_2_test.append(at_idx_minus_2)

In [ ]:
set(list_index_minus_2)

In [ ]:
# correct

In [ ]:
# now make the class token indices 103

In [ ]:
for idx, val in enumerate(class_tokens_indices_test_list):
    inputs_test['input_ids'][idx][val] = 103

In [ ]:
# now check if they are 103

In [ ]:
unsontroi_check_list = []

for idx, val in enumerate(class_tokens_indices_test_list):
    at_idx_minus_2 = inputs_test['input_ids'][idx][val].item()
    unsontroi_check_list.append(at_idx_minus_2)

In [ ]:
set(unsontroi_check_list)

In [ ]:
# correct

In [ ]:
# class PeDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings
#     def __getitem__(self, idx):
#         return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#     def __len__(self):
#         return len(self.encodings.input_ids)

In [ ]:
test_dataset = PeDataset(inputs_test)

In [ ]:
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True)
# maybe we don't need this.

## get the model

In [143]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [144]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [145]:
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [146]:
# let's see with a simple trainer now and then we will try the fancy trainer.

### training

In [147]:
from transformers import TrainingArguments

In [148]:
import gc

In [149]:
gc.collect()
torch.cuda.empty_cache()

In [150]:
NB_EPOCHS = 10
BATCH_SIZE = 8

In [151]:
training_args = TrainingArguments(
    
    # output
    output_dir='output',          
    
    # params
    num_train_epochs=NB_EPOCHS,               # nb of epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,    # cf. paper Sun et al.
    learning_rate=1e-5,#2e-5,                 # cf. paper Sun et al.
#     warmup_steps=500,                         # number of warmup steps for learning rate scheduler
    warmup_ratio=0.1,                         # cf. paper Sun et al.
    weight_decay=0.01,                        # strength of weight decay
    
    # eval
    evaluation_strategy="epoch",              # cf. paper Sun et al.
    #eval_steps=20,                            # cf. paper Sun et al.
    #eval_accumulation_steps = BATCH_SIZE,
    # fp16_full_eval = True,
    
    # clean the tensor board folder to see the loss properly.
    
    # log
    logging_dir="/notebooks/Prompting/results",  
    logging_strategy='steps',
    logging_steps=100,
    
    # save
    save_strategy='epoch',
    save_total_limit=2,
    # save_steps=20, # default 500
    load_best_model_at_end=True,              # cf. paper Sun et al.
    metric_for_best_model='eval_loss' 
    # metric_for_best_model='f1'
)

In [152]:
# from transformers import TrainingArguments

# args = TrainingArguments(
#     output_dir = 'out',
#     per_device_train_batch_size=16,
#     num_train_epochs=2
# )

In [153]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
    #compute_metrics=compute_metrics
)

In [154]:
trainer.train()

***** Running training *****
  Num examples = 4237
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5300
/tmp/ipykernel_399/1625588428.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,0.011300,0.004815
2,0.003800,0.002503
3,0.002800,0.002267
4,0.002300,0.002636
5,0.002000,0.002023
6,0.002000,0.002023
7,0.001900,0.001988
8,0.001900,0.001979
9,0.001900,0.001966
10,0.001700,0.001945


***** Running Evaluation *****
  Num examples = 472
  Batch size = 8
Saving model checkpoint to output/checkpoint-530
Configuration saved in output/checkpoint-530/config.json
Model weights saved in output/checkpoint-530/pytorch_model.bin
/tmp/ipykernel_399/1625588428.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 472
  Batch size = 8
Saving model checkpoint to output/checkpoint-1060
Configuration saved in output/checkpoint-1060/config.json
Model weights saved in output/checkpoint-1060/pytorch_model.bin
/tmp/ipykernel_399/1625588428.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.t

TrainOutput(global_step=5300, training_loss=0.24367328433214494, metrics={'train_runtime': 2939.6763, 'train_samples_per_second': 14.413, 'train_steps_per_second': 1.803, 'total_flos': 1.1151987918336e+16, 'train_loss': 0.24367328433214494, 'epoch': 10.0})

In [155]:
trainer.save_model("pe_mask_model_prompted_rep_2_joint")

Saving model checkpoint to pe_mask_model_prompted_rep_2_joint
Configuration saved in pe_mask_model_prompted_rep_2_joint/config.json
Model weights saved in pe_mask_model_prompted_rep_2_joint/pytorch_model.bin


In [156]:
# new idea: take this model and finetune the PE dataset on it.
# so instead of loading a pre-trained BERT, load this instead.

### Pipeline

In [156]:
saved_model = BertForMaskedLM.from_pretrained("/notebooks/Prompting/notebooks/mask_joint_tasks/pe_mask_model_prompted_rep_2_joint")

loading configuration file /notebooks/Prompting/notebooks/mask_joint_tasks/pe_mask_model_prompted_rep_2_joint/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /notebooks/Prompting/notebooks/mask_joint_tasks/pe_mask_model_prompted_rep_2_joint/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMas

In [157]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [158]:
from transformers import pipeline

In [159]:
classifier = pipeline("fill-mask", model=saved_model, tokenizer=tokenizer)

In [160]:
classifier.model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [161]:
# we need to do two targets, two masked reps and so forth, one for each task.
# the things that are being done here but for both tasks separately

#### for component class

In [163]:
# first check which index class is when we split

In [166]:
df_test = df[df.split == 'TEST'].reset_index(drop=True)

In [172]:
df_test['corrected_representation_2'][0].split()[-5]

'Stance.'

In [173]:
targets = ['stance', 'claim', 'premise']

In [174]:
# ok now see how to do this for the whole list of test components

In [175]:
# but first we must replace the last token by [MASK]!

In [176]:
def get_masked_components(x):
    
    prompted_rep = x.corrected_representation_2
    
    last_token = prompted_rep.split()[-5]
    

    new_rep = prompted_rep.replace(last_token, "[MASK].")

    
    return new_rep

In [177]:
df_test

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_2,prompted_representation_3,prompted_representation_2_correct,prompted_representation_2_joint,corrected_representation_2
0,essay004,T1,MajorClaim 262 376,this industry has affected the cultural attrib...,MajorClaim,MajorClaim,[],,Linked,TEST,...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, DET...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
1,essay004,T2,MajorClaim 1663 1758,the tourism has created threatening pressure o...,MajorClaim,MajorClaim,[],,Linked,TEST,...,1.00,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, VER...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
2,essay004,T3,Claim 179 239,the tourism bring large profit for the destina...,Claim,Claim,[],Attack,NotLinked,TEST,...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, VERB, ADJ, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
3,essay004,T4,Claim 953 1031,international tourism can create negative impa...,Claim,Claim,[],Support,Linked,TEST,...,0.50,0,Topic: International tourism is now more commo...,"Part Of Speech tags: ADJ, NOUN, VERB, VERB, AD...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
4,essay004,T5,Claim 1578 1624,tourism has threatened the nature environments,Claim,Claim,[],Support,Linked,TEST,...,0.75,0,Topic: International tourism is now more commo...,"Part Of Speech tags: NOUN, AUX, VERB, DET, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,essay398,T12,Claim 1484 1589,universities should encourage more girls to ch...,Claim,Claim,[],Support,Linked,TEST,...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: NOUN, VERB, VERB, ADJ, NO...",Topic: We can not forcedly put the same number...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
1256,essay398,T13,Premise 1595 1648,this could avoid imbalance of gender in some s...,Premise,Premise,[],Support,NotLinked,TEST,...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: DET, VERB, VERB, NOUN, AD...",Topic: We can not forcedly put the same number...,Which of these cho

In [178]:
df_test['masked_representation_2_for_pipeline_joint'] = df_test.apply(lambda x: get_masked_components(x), axis=1)

In [179]:
df_test['masked_representation_2_for_pipeline_joint'][10]

'Which of these choices best describes the following component? "Stance", "Claim" or "Premise". Is this component "Linked" or "NotLinked"? Component: Topic: International tourism is now more common than ever before, Sentence: For this reason, many marine lives have been endangered, in the extremes part of the reef become uninhabitable for these marine species, First or last in essay: No, First in paragraph: No, Last in paragraph: No, In in introduction: No, Is in conclusion: No. Part Of Speech tags: ADJ, ADJ, NOUN, AUX, AUX, VERB, PUNCT, ADP, DET, NOUN, NOUN, ADP, DET, NOUN, VERB, ADJ, ADP, DET, ADJ, NOUN. [MASK]. This component is Linked.'

In [163]:
# ok now do a sanity check to see that they are done correctly

In [180]:
last_tokens_mask_list = []

In [181]:
def get_last_token(x):
    
    prompted_rep = x.masked_representation_2_for_pipeline_joint
    
    last_token = prompted_rep.split()[-5]
    
    last_tokens_mask_list.append(last_token)

In [182]:
df_test.apply(lambda x: get_last_token(x), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
1255    None
1256    None
1257    None
1258    None
1259    None
Length: 1260, dtype: object

In [183]:
last_tokens_mask_list

['[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[M

In [184]:
set(last_tokens_mask_list)

{'[MASK].'}

In [169]:
###### YASSSSS SIRRRRRRRRRRR # correct

In [170]:
# ok now see how we can give all these 1260 statements to the pipeline

In [185]:
test_components_list = df_test['masked_representation_2_for_pipeline_joint'][:].tolist()

In [186]:
len(test_components_list)

1260

In [187]:
# ok now give it to the pipeline

In [188]:
classifier_output = classifier(test_components_list, targets=targets)

Disabling tokenizer parallelism, we're using DataLoader multithreading already


In [189]:
classifier_output[0][0]['token']

11032

In [190]:
predicted_labels_test = []

for idx in range(1260):
    
    predicted_token = classifier_output[idx][0]['token']
    predicted_labels_test.append(predicted_token)

In [191]:
len(predicted_labels_test)

1260

In [192]:
# sanity check
set(predicted_labels_test)

{4366, 11032, 18458}

In [193]:
# ok correct

In [194]:
test_labels_str = df_test.label_ComponentType[:].tolist()

In [195]:
test_labels_str

['MajorClaim',
 'MajorClaim',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'MajorClaim',
 'MajorClaim',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Claim',
 'MajorClaim',
 'MajorClaim',
 'Claim',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'MajorClaim',
 'MajorClaim',
 'Claim',
 'Premise',
 'Premise',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'MajorClaim',
 'MajorClaim',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'MajorClaim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',

In [196]:
set(test_labels_str)

{'Claim', 'MajorClaim', 'Premise'}

In [197]:
test_labels_int = []

In [198]:
def get_int_labels():
    
    for idx, val in enumerate(test_labels_str):
        
        if val == 'MajorClaim':
            
            test_labels_int.append(11032)
            
        elif val == 'Claim':
            
            test_labels_int.append(4366)
            
        elif val == 'Premise':
            
            test_labels_int.append(18458)

In [199]:
get_int_labels()

In [200]:
test_labels_int

[11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 4366,
 11032,
 11032,
 4366,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 18458,
 18458,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 4366,
 11032,
 4366,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 18458,
 18458,
 11032,
 11032,
 4366,
 4366,
 4366,
 18458,
 18458,
 18458,
 18458,
 11032,
 4366,
 11032,


In [201]:
len(test_labels_int)

1260

In [202]:
set(test_labels_int)

{4366, 11032, 18458}

In [203]:
# now do the classification report

In [204]:
from sklearn.metrics import classification_report

In [205]:
print(classification_report(test_labels_int, predicted_labels_test, digits=3))

              precision    recall  f1-score   support

        4366      0.655     0.775     0.710       302
       11032      0.852     0.830     0.841       153
       18458      0.936     0.877     0.906       805

    accuracy                          0.847      1260
   macro avg      0.815     0.827     0.819      1260
weighted avg      0.859     0.847     0.851      1260



#### link identifiction

In [163]:
# first check which index class is when we split

In [166]:
# df_test = df[df.split == 'TEST'].reset_index(drop=True)

In [206]:
df_test['corrected_representation_2'][0].split()[-1]

'Linked.'

In [207]:
targets = ['linked', 'separate']

In [208]:
# ok now see how to do this for the whole list of test components

In [209]:
# but first we must replace the last token by [MASK]!

In [210]:
def get_masked_components(x):
    
    prompted_rep = x.corrected_representation_2
    
    last_token = prompted_rep.split()[-1]
    

    new_rep = prompted_rep.replace(last_token, "[MASK].")

    
    return new_rep

In [211]:
df_test

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_2,prompted_representation_3,prompted_representation_2_correct,prompted_representation_2_joint,corrected_representation_2,masked_representation_2_for_pipeline_joint
0,essay004,T1,MajorClaim 262 376,this industry has affected the cultural attrib...,MajorClaim,MajorClaim,[],,Linked,TEST,...,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, DET...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
1,essay004,T2,MajorClaim 1663 1758,the tourism has created threatening pressure o...,MajorClaim,MajorClaim,[],,Linked,TEST,...,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, VER...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
2,essay004,T3,Claim 179 239,the tourism bring large profit for the destina...,Claim,Claim,[],Attack,NotLinked,TEST,...,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, VERB, ADJ, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
3,essay004,T4,Claim 953 1031,international tourism can create negative impa...,Claim,Claim,[],Support,Linked,TEST,...,0,Topic: International tourism is now more commo...,"Part Of Speech tags: ADJ, NOUN, VERB, VERB, AD...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
4,essay004,T5,Claim 1578 1624,tourism has threatened the nature environments,Claim,Claim,[],Support,Linked,TEST,...,0,Topic: International tourism is now more commo...,"Part Of Speech tags: NOUN, AUX, VERB, DET, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,essay398,T12,Claim 1484 1589,universities should encourage more girls to ch...,Claim,Claim,[],Support,Linked,TEST,...,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: NOUN, VERB, VERB, ADJ, NO...",Topic: We can not forcedly put the same number...,Which of these choices best describes the foll...,"How is the component best described?: ""MajorCl...",Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
1256,es

In [212]:
df_test['masked_representation_2_for_pipeline_joint'] = df_test.apply(lambda x: get_masked_components(x), axis=1)

In [213]:
df_test['masked_representation_2_for_pipeline_joint'][10]

'Which of these choices best describes the following component? "Stance", "Claim" or "Premise". Is this component "Linked" or "NotLinked"? Component: Topic: International tourism is now more common than ever before, Sentence: For this reason, many marine lives have been endangered, in the extremes part of the reef become uninhabitable for these marine species, First or last in essay: No, First in paragraph: No, Last in paragraph: No, In in introduction: No, Is in conclusion: No. Part Of Speech tags: ADJ, ADJ, NOUN, AUX, AUX, VERB, PUNCT, ADP, DET, NOUN, NOUN, ADP, DET, NOUN, VERB, ADJ, ADP, DET, ADJ, NOUN. Premise. This component is [MASK].'

In [214]:
# ok now do a sanity check to see that they are done correctly

In [222]:
last_tokens_mask_list = []

In [223]:
def get_last_token(x):
    
    prompted_rep = x.masked_representation_2_for_pipeline_joint
    
    last_token = prompted_rep.split()[-1]
    
    last_tokens_mask_list.append(last_token)

In [224]:
df_test.apply(lambda x: get_last_token(x), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
1255    None
1256    None
1257    None
1258    None
1259    None
Length: 1260, dtype: object

In [225]:
last_tokens_mask_list

['[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[M

In [226]:
set(last_tokens_mask_list)

{'[MASK].'}

In [227]:
###### YASSSSS SIRRRRRRRRRRR # correct

In [228]:
# ok now see how we can give all these 1260 statements to the pipeline

In [229]:
test_components_list = df_test['masked_representation_2_for_pipeline_joint'][:].tolist()

In [230]:
len(test_components_list)

1260

In [231]:
# ok now give it to the pipeline

In [232]:
classifier_output = classifier(test_components_list, targets=targets)

In [233]:
classifier_output[0][0]['token']

3584

In [234]:
predicted_labels_test = []

for idx in range(1260):
    
    predicted_token = classifier_output[idx][0]['token']
    predicted_labels_test.append(predicted_token)

In [235]:
len(predicted_labels_test)

1260

In [236]:
# sanity check
set(predicted_labels_test)

{3584, 5799}

In [237]:
# ok correct

In [238]:
test_labels_str = df_test.label_LinkedNotLinked[:].tolist()

In [239]:
test_labels_str

['Linked',
 'Linked',
 'NotLinked',
 'Linked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'Linked',
 'Linked',
 'Linked',
 'Linked',
 'Linked',
 'NotLinked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'Linked',
 'NotLinked',
 'Linked',
 'Linked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'Linked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'Linked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'Linked',
 'NotLinked',
 'Linked',
 'Linked',
 'NotLinked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'Linked',
 'Linked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',

In [240]:
set(test_labels_str)

{'Linked', 'NotLinked'}

In [241]:
test_labels_int = []

In [242]:
def get_int_labels():
    
    for idx, val in enumerate(test_labels_str):
        
        if val == 'Linked':
            
            test_labels_int.append(5799)
            
        elif val == 'NotLinked':
            
            test_labels_int.append(3584)

In [243]:
get_int_labels()

In [244]:
test_labels_int

[5799,
 5799,
 3584,
 5799,
 5799,
 3584,
 3584,
 3584,
 3584,
 3584,
 5799,
 5799,
 5799,
 5799,
 5799,
 5799,
 3584,
 5799,
 3584,
 3584,
 3584,
 3584,
 3584,
 5799,
 5799,
 3584,
 5799,
 5799,
 5799,
 3584,
 3584,
 5799,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 5799,
 3584,
 3584,
 3584,
 5799,
 5799,
 5799,
 3584,
 3584,
 5799,
 5799,
 5799,
 3584,
 3584,
 3584,
 3584,
 5799,
 3584,
 3584,
 5799,
 3584,
 3584,
 5799,
 5799,
 3584,
 5799,
 5799,
 3584,
 5799,
 3584,
 3584,
 3584,
 3584,
 5799,
 5799,
 5799,
 5799,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 5799,
 3584,
 5799,
 3584,
 5799,
 5799,
 5799,
 5799,
 3584,
 3584,
 3584,
 5799,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 5799,
 5799,
 3584,
 5799,
 5799,
 3584,
 3584,
 3584,
 5799,
 3584,
 3584,
 5799,
 5799,
 5799,
 5799,
 5799,
 3584,
 3584,
 3584,
 3584,
 5799,
 3584,
 5799,
 5799,
 3584,
 3584,
 3584,
 3584,
 5799,
 3584,
 3584,
 3584,
 5799,
 5799,
 5799,
 5799,
 5799,

In [245]:
len(test_labels_int)

1260

In [246]:
set(test_labels_int)

{3584, 5799}

In [247]:
# now do the classification report

In [248]:
from sklearn.metrics import classification_report

In [249]:
print(classification_report(test_labels_int, predicted_labels_test, digits=3))

              precision    recall  f1-score   support

        3584      0.801     0.849     0.825       744
        5799      0.762     0.696     0.727       516

    accuracy                          0.787      1260
   macro avg      0.782     0.773     0.776      1260
weighted avg      0.785     0.787     0.785      1260

